In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from scipy.optimize import curve_fit
from collections import defaultdict
import ast
import re
from itertools import combinations
import itertools
from matplotlib import font_manager
import matplotlib.dates as mdates
from sklearn.neighbors import NearestNeighbors
import random
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances
import seaborn as sns
from kneed import KneeLocator
from matplotlib.ticker import MaxNLocator

# # 指定字体
# font_path = "C:/Windows/Fonts/simsun.ttc"  # SimSun字体路径
# font_prop = font_manager.FontProperties(fname=font_path)

# plt.rcParams["font.sans-serif"] = [font_prop.get_name()]
# plt.rcParams["axes.unicode_minus"] = False  # 解决负号显示问题
# plt.rcParams["font.size"] = 10.5  # 设置五号字体大小

In [ ]:
# 设置全局字体为 Times New Roman
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'

In [ ]:
# 设置xtick和ytick的方向：in、out、inout
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

删除非带货直播

In [ ]:
file_paths = ["D:/研究生/毕业论文/数据/主播数据_3个月/livelinks_1000(2024.06.08-2024.09.05).csv", 
             "D:/研究生/毕业论文/数据/主播数据_3个月/livelinks_500-1000(2024.06.08-2024.09.05).csv",
             "D:/研究生/毕业论文/数据/主播数据_3个月/livelinks_100-500(2024.06.08-2024.09.05).csv",
            "D:/研究生/毕业论文/数据/主播数据_3个月/livelinks_10-100(2024.06.08-2024.09.05).csv"
             ]

In [ ]:
# 读取xlsx文件
df = pd.read_csv('D:/研究生/毕业论文/数据/主播数据_3个月/livelinks_1000(2024.06.08-2024.09.05).csv')
# 筛选is_take_product为FALSE的行。gbk读取为布尔值，而不是字符串;如果不是gbk，则是字符串。
filtered_df = df[df['is_take_product'] == 'False']
# 获取要删除的room_id列表
room_ids_to_delete = filtered_df['room_id']
room_ids_to_delete = room_ids_to_delete.str.replace('\t', '')
data_folder = 'D:/研究生/毕业论文/数据/主播数据_3个月/meichangzhibodata_1000(2024.06.08-2024.09.05)'
# 遍历文件夹中的文件
for filename in os.listdir(data_folder):
    # 提取文件名的前缀部分（去掉文件扩展名）
    file_prefix = os.path.splitext(filename)[0]  # 获取不含扩展名的文件名前缀

    # 检查提取的文件前缀是否在room_ids_to_delete中
    if file_prefix in room_ids_to_delete.values:
        # 构建完整的文件路径
        file_path = os.path.join(data_folder, filename)
        try:
            # 删除文件
            os.remove(file_path)
            print(f"Deleted file: {file_path}")
        except Exception as e:
            print(f"Error deleting file {file_path}: {e}")
    else:
        print(f"Skipped file: {filename} (room_id not in list)")

In [ ]:
len(room_ids_to_delete)

# 1.确认数据范围

In [ ]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
# file_paths = ["D:/研究生/毕业论文/数据/主播数据_3个月/jieguo_1000(2024.06.08-2024.09.05)/info.xlsx", 
#              "D:/研究生/毕业论文/数据/主播数据_3个月/jieguo_500-1000(2024.06.08-2024.09.05)/info.xlsx",
#              "D:/研究生/毕业论文/数据/主播数据_3个月/jieguo_100-500(2024.06.08-2024.09.05)/info.xlsx",
#             "D:/研究生/毕业论文/数据/主播数据_3个月/jieguo_10-100(2024.06.08-2024.09.05)/info.xlsx"
#              ]
file_paths = [
             "D:/研究生/论文/毕业论文/数据/主播数据_3个月/jieguo_100-500(2024.06.08-2024.09.05)/info.xlsx",
            "D:/研究生/论文/毕业论文/数据/主播数据_3个月/jieguo_10-100(2024.06.08-2024.09.05)/info.xlsx"
             ]


In [ ]:
# 创建一个空DataFrame用于存储所有数据
df_all = pd.DataFrame()
for file_path in file_paths:
    # 读取xlsx文件
    df = pd.read_excel(file_path)
    # 合并数据
    df_all = pd.concat([df_all, df], ignore_index=True)

# 将 begin_time 和 room_finish_time 列的时间戳转换为 pandas 的日期时间格式
df_all['begin_time'] = pd.to_datetime(df_all['begin_time'], unit='s')
df_all['room_finish_time'] = pd.to_datetime(df_all['room_finish_time'], unit='s')

# 将日期时间列的时区从 UTC 转换为北京时间
df_all['begin_time'] = df_all['begin_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
df_all['room_finish_time'] = df_all['room_finish_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')

# 提取日期作为新的列
df_all['date'] = df_all['begin_time'].dt.date

In [ ]:
# 读取三个xlsx文件，假设文件名分别为file1.xlsx, file2.xlsx, file3.xlsx
# file_multi_paths = ["D:/研究生/毕业论文/数据/主播数据_3个月/jieguo_1000(2024.06.08-2024.09.05)/multi.xlsx", 
#              "D:/研究生/毕业论文/数据/主播数据_3个月/jieguo_500-1000(2024.06.08-2024.09.05)/multi.xlsx",
#              "D:/研究生/毕业论文/数据/主播数据_3个月/jieguo_100-500(2024.06.08-2024.09.05)/multi.xlsx",
#               "D:/研究生/毕业论文/数据/主播数据_3个月/jieguo_10-100(2024.06.08-2024.09.05)/multi.xlsx"     ]
file_multi_paths = [
             "D:/研究生/论文/毕业论文/数据/主播数据_3个月/jieguo_100-500(2024.06.08-2024.09.05)/multi.xlsx",
              "D:/研究生/论文/毕业论文/数据/主播数据_3个月/jieguo_10-100(2024.06.08-2024.09.05)/multi.xlsx"     ]

In [ ]:
# 读取multi.xlsx文件
multi_df = pd.concat([pd.read_excel(file_path) for file_path in file_multi_paths])

In [ ]:
# 根据直播ID合并两个DataFrame
merged_df = pd.merge(df_all, multi_df, on='room_id', how='inner')

In [ ]:
# 创建一个空列表，用于存储格式有问题的 room_id
invalid_room_ids = []

# 遍历 merged_df 中的每一行
for index, row in merged_df.iterrows():
    # 获取当前行的 room_id 和 charts 列数据
    room_id = row['room_id']
    charts_data = row['charts']
    
    # 将单引号替换为双引号
    charts_data = charts_data.replace("'", "\"")
    
    try:
        # 使用 json.loads() 将字符串解析为 JSON 对象
        charts_list = json.loads(charts_data)
    except json.JSONDecodeError:
        # 如果解析失败，则记录该行的 room_id
        invalid_room_ids.append(room_id)
# 打印格式有问题的 room_id
print("格式有问题的 room_id：", len(invalid_room_ids))
print("格式有问题的 room_id：", invalid_room_ids)

In [ ]:
# 删除格式有问题的 room_id 对应的行
merged_df = merged_df[~merged_df['room_id'].isin(invalid_room_ids)]

In [ ]:
# 创建一个空列表，用于存储展开后的数据
expanded_data_list = []

# 遍历 merged_df 中的每一行
for index, row in merged_df.iterrows():
    # 获取当前行的 room_id 和 charts 列数据
    room_id = row['room_id']
    # follower = row['follower_count']
    charts_data = row['charts']
    
    # 将单引号替换为双引号
    charts_data = charts_data.replace("'", "\"")
    
    # 使用 json.loads() 将字符串解析为 JSON 对象
    charts_list = json.loads(charts_data)
    
    # 遍历每个直播间的 charts 数据
    for chart in charts_list:
        # 将每个字典添加到列表中，并添加 room_id
        chart['room_id'] = room_id
        # chart['follower'] = follower
        expanded_data_list.append(chart)


In [ ]:
# 将列表转换为 DataFrame
expanded_data = pd.DataFrame(expanded_data_list)

# 将 time_node 列转换为日期时间类型
expanded_data['time_node'] = pd.to_datetime(expanded_data['time_node'], unit='s')

expanded_data['time_node'] = expanded_data['time_node'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')

# 根据 time_node 列提取日期
expanded_data['date'] = expanded_data['time_node'].dt.date

# 提取小时
expanded_data['hour'] = expanded_data['time_node'].dt.hour

# 提取分钟
expanded_data['minute'] = expanded_data['time_node'].dt.minute

In [ ]:
# 列出需要增加的列名
columns_to_increment = ['real_time_user', 'leave_user', 'user_count']

# 将这些列的数据全部加 1
expanded_data[columns_to_increment] = expanded_data[columns_to_increment] + 1


In [ ]:
def plot_feature_changes(filtered_data, time_interval='5T', save_path=None):
    """
    绘制每个等级的特征变化图，根据指定的时间间隔。

    Parameters:
    - filtered_data (pd.DataFrame): 包含数据的 DataFrame。
    - time_interval (str): 时间间隔，格式为 '5T', '10T', '30T' 等。
    - save_path (str): 图像保存路径。
    """
    # 创建数据副本，避免链式赋值问题
    filtered_data_n = filtered_data.copy()
    # 使用 .loc 方法进行赋值，避免链式赋值警告
    filtered_data_n.loc[:, 'time_period'] = filtered_data_n['minute'] // int(time_interval[:-1]) * int(time_interval[:-1])

    # 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
    average_features = filtered_data_n.groupby(['date', 'hour', 'time_period', 'room_id'])[['real_time_user', 'leave_user', 'user_count']].mean().reset_index()

    # 将同等级的直播间进行求和
    summed_features = average_features.groupby(['date', 'hour', 'time_period'])[['real_time_user', 'leave_user', 'user_count']].sum().reset_index()

    # 按月计算每个特征的平均值
    monthly_averages = summed_features.groupby(['hour', 'time_period']).agg({
        'real_time_user': 'mean',
        'leave_user': 'mean',
        'user_count': 'mean'
    }).reset_index()

    # 将 'time_period' 转换为实际时间格式
    monthly_averages['time_period_str'] = monthly_averages['time_period'].astype(int).astype(str).str.zfill(2) + ':00'
    monthly_averages['time_period'] = pd.to_datetime(monthly_averages['hour'].astype(str) + ':' + monthly_averages['time_period_str'], format='%H:%M:%S')


    # 创建图形
    fig, ax = plt.subplots(figsize=(20, 4))

    # 绘制特征数据
    ax.plot(monthly_averages['time_period'], monthly_averages['real_time_user'], marker='o', label='进场用户', color='#4169e0')
    ax.plot(monthly_averages['time_period'], monthly_averages['leave_user'], marker='x', label='离线用户', color='#ff0000')
    ax.plot(monthly_averages['time_period'], monthly_averages['user_count'], marker='s', label='在线用户', color='#00cdd0')

    # 设置标题和标签
    # ax.set_title(f'每隔{int(time_interval[:-1])}分钟的特征变化', fontweight='bold')
    ax.set_ylabel('数量', fontweight='bold', fontproperties=font_prop,fontsize=12)
    ax.legend()

    # 设置 x 轴标签
    ax.set_xlabel('时间区间', fontproperties=font_prop,fontsize=12)

    # 设置 x 轴刻度
    # 设置 x 轴刻度为每小时一个刻度
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

    # # 自动旋转 x 轴刻度标签
    # plt.setp(ax.get_xticklabels(), rotation=45)

    # 调整布局
    plt.tight_layout()

    # # 保存图为文件
    plt.savefig(save_path)

    # 显示图形
    plt.show()


In [ ]:
def plot_feature_changes(filtered_data, time_interval='5T', save_path=None):
    """
    绘制每个等级的特征变化图，根据指定的时间间隔。

    Parameters:
    - filtered_data (pd.DataFrame): 包含数据的 DataFrame。
    - time_interval (str): 时间间隔，格式为 '5T', '10T', '30T' 等。
    - save_path (str): 图像保存路径。
    """
    # 创建数据副本，避免链式赋值问题
    filtered_data_n = filtered_data.copy()
    # 使用 .loc 方法进行赋值，避免链式赋值警告
    filtered_data_n.loc[:, 'time_period'] = filtered_data_n['minute'] // int(time_interval[:-1]) * int(time_interval[:-1])

    # 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
    average_features = filtered_data_n.groupby(['date', 'hour', 'time_period', 'room_id'])[['real_time_user', 'leave_user', 'user_count']].mean().reset_index()

    # 将同等级的直播间进行求和
    summed_features = average_features.groupby(['date', 'hour', 'time_period'])[['real_time_user', 'leave_user', 'user_count']].sum().reset_index()

    # 按月计算每个特征的平均值
    monthly_averages = summed_features.groupby(['hour', 'time_period']).agg({
        'real_time_user': 'mean',
        'leave_user': 'mean',
        'user_count': 'mean'
    }).reset_index()

    # 将 'time_period' 转换为实际时间格式
    monthly_averages['time_period_str'] = monthly_averages['time_period'].astype(int).astype(str).str.zfill(2) + ':00'
    monthly_averages['time_period'] = pd.to_datetime(monthly_averages['hour'].astype(str) + ':' + monthly_averages['time_period_str'], format='%H:%M:%S')

    # 创建图形
    fig, ax = plt.subplots(figsize=(16, 6))

    # 绘制特征数据
    ax.plot(monthly_averages['time_period'], monthly_averages['real_time_user'], marker='o', label='进场用户', color='#4169e0')
    ax.plot(monthly_averages['time_period'], monthly_averages['leave_user'], marker='x', label='离线用户', color='#ff0000')
    ax.plot(monthly_averages['time_period'], monthly_averages['user_count'], marker='s', label='在线用户', color='#00cdd0')

    # 设置标题和标签
    ax.set_ylabel('数量', fontweight='bold', fontproperties=font_prop, fontsize=20)
    ax.legend()

    # 设置 x 轴标签
    ax.set_xlabel('时间区间', fontproperties=font_prop, fontsize=20)

    # 设置 x 轴刻度
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=4))  # 每两个小时一个刻度
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

    # 增加字体大小
    for label in ax.get_xticklabels():
        label.set_fontsize(20)  # 设置x轴刻度字体大小
    for label in ax.get_yticklabels():
        label.set_fontsize(20)  # 设置y轴刻度字体大小
    
    ax.yaxis.set_major_locator(MaxNLocator(integer=True, nbins=5))  # 设置y轴刻度数量
    
    # 调整图例
    ax.legend(fontsize=20, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=3)  # 在正上方

    plt.tight_layout()


    # 保存图为文件
    plt.savefig(save_path)

    # 显示图形
    plt.show()


In [ ]:
def plot_feature_changes_1(filtered_data, time_interval='5T', save_path=None):
    """
    绘制每个等级的特征变化图，根据指定的时间间隔。

    Parameters:
    - filtered_data (pd.DataFrame): 包含数据的 DataFrame。
    - time_interval (str): 时间间隔，格式为 '5T', '10T', '30T' 等。
    - save_path (str): 图像保存路径。
    """
    # 创建数据副本，避免链式赋值问题
    filtered_data_n = filtered_data.copy()
    # 使用 .loc 方法进行赋值，避免链式赋值警告
    filtered_data_n.loc[:, 'time_period'] = filtered_data_n['minute'] // int(time_interval[:-1]) * int(time_interval[:-1])

    # 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
    average_features = filtered_data_n.groupby(['date', 'hour', 'time_period', 'room_id'])[['real_time_user', 'leave_user', 'user_count']].mean().reset_index()

    # 将同等级的直播间进行求和
    summed_features = average_features.groupby(['date', 'hour', 'time_period'])[['real_time_user', 'leave_user', 'user_count']].sum().reset_index()

    # 按月计算每个特征的平均值
    monthly_averages = summed_features.groupby(['hour', 'time_period']).agg({
        'real_time_user': 'mean',
        'leave_user': 'mean',
        'user_count': 'mean'
    }).reset_index()

    # 将 'time_period' 转换为实际时间格式
    monthly_averages['time_period_str'] = monthly_averages['time_period'].astype(int).astype(str).str.zfill(2) + ':00'
    monthly_averages['time_period'] = pd.to_datetime(monthly_averages['hour'].astype(str) + ':' + monthly_averages['time_period_str'], format='%H:%M:%S')

    # 创建图形
    fig, ax = plt.subplots(figsize=(16, 6))

    # 绘制特征数据
    ax.plot(monthly_averages['time_period'], monthly_averages['real_time_user'], marker='o', label='New Users', color='#4169e0')
    ax.plot(monthly_averages['time_period'], monthly_averages['leave_user'], marker='x', label='Offline Users', color='#ff0000')
    ax.plot(monthly_averages['time_period'], monthly_averages['user_count'], marker='s', label='Online Users', color='#00cdd0')

    # 设置标题和标签
    ax.set_ylabel('Quantity', fontsize=20)
    ax.legend()

    # 设置 x 轴标签
    ax.set_xlabel('Time Interval', fontsize=20)

    # 设置 x 轴刻度
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=4))  # 每两个小时一个刻度
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

    # 增加字体大小
    for label in ax.get_xticklabels():
        label.set_fontsize(20)  # 设置x轴刻度字体大小
    for label in ax.get_yticklabels():
        label.set_fontsize(20)  # 设置y轴刻度字体大小
    
    ax.yaxis.set_major_locator(MaxNLocator(integer=True, nbins=5))  # 设置y轴刻度数量
    
    # 调整图例
    ax.legend(fontsize=20, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=3)  # 在正上方

    plt.tight_layout()


    # 保存图为文件
    plt.savefig(save_path)

    # 显示图形
    plt.show()

In [ ]:
plot_feature_changes(expanded_data, time_interval='1T', save_path='D:/研究生/毕业论文/代码/网络构建/直播平台网络模型/图片/时间范围_1.svg')

In [ ]:
plot_feature_changes(expanded_data, time_interval='2T', save_path='D:/研究生/毕业论文/代码/网络构建/直播平台网络模型/图片/时间范围_2.svg')

In [ ]:
plot_feature_changes(expanded_data, time_interval='3T', save_path='D:/研究生/毕业论文/草稿/图片/时间范围_3.svg')

In [ ]:
plot_feature_changes(expanded_data, time_interval='10T')

In [ ]:
plot_feature_changes(expanded_data, time_interval='60T')

In [ ]:
plot_feature_changes_1(expanded_data, time_interval='3T', save_path='D:/研究生/论文/期刊论文/小论文1/草稿/图片/time_interval_3.eps')

确认好数据的范围为：19：00-01：00.

# 2.确认窗口大小

In [ ]:
# 进行数据筛选，只需要晚上时间段的直播
filtered_data = expanded_data[expanded_data['hour'].isin([19, 20, 21, 22, 23, 0])]

In [ ]:
def plot_feature_changes_part(filtered_data, time_interval='5T', save_path=None):
    """
    绘制每个等级的特征变化图，根据指定的时间间隔。

    Parameters:
    - filtered_data (pd.DataFrame): 包含数据的 DataFrame。
    - time_interval (str): 时间间隔，格式为 '5T', '10T', '30T' 等。
    - save_path (str): 图像保存路径。
    """
    # 创建数据副本，避免链式赋值问题
    filtered_data_n = filtered_data.copy()
    # 使用 .loc 方法进行赋值，避免链式赋值警告
    filtered_data_n.loc[:, 'time_period'] = filtered_data_n['minute'] // int(time_interval[:-1]) * int(time_interval[:-1])

    # 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
    average_features = filtered_data_n.groupby(['date', 'hour', 'time_period', 'room_id'])[['real_time_user', 'leave_user', 'user_count']].mean().reset_index()

    # 将同等级的直播间进行求和
    summed_features = average_features.groupby(['date', 'hour', 'time_period'])[['real_time_user', 'leave_user', 'user_count']].sum().reset_index()

    # 按月计算每个特征的平均值
    monthly_averages = summed_features.groupby(['hour', 'time_period']).agg({
        'real_time_user': 'mean',
        'leave_user': 'mean',
        'user_count': 'mean'
    }).reset_index()

    # 你想要的小时顺序
    desired_order = [19, 20, 21, 22, 23, 0, 1]

    # 将 hour 列转换为 Categorical 类型，并指定排序顺序
    monthly_averages['hour'] = pd.Categorical(monthly_averages['hour'], categories=desired_order, ordered=True)

    # 根据 hour 列和 time_period 列对 DataFrame 进行排序
    monthly_averages_sorted = monthly_averages.sort_values(['hour', 'time_period'])

    # 将 'time_period' 转换为实际时间格式
    monthly_averages_sorted['time_period_str'] = monthly_averages_sorted['time_period'].astype(int).astype(str).str.zfill(2) + ':00'
    monthly_averages_sorted['time_period'] = pd.to_datetime(monthly_averages_sorted['hour'].astype(str) + ':' + monthly_averages_sorted['time_period_str'], format='%H:%M:%S')

    # 对于 hour 为 0 或 1，日期加一天
    monthly_averages_sorted.loc[monthly_averages_sorted['hour'].isin([0, 1]), 'time_period'] = \
        monthly_averages_sorted['time_period'] + pd.DateOffset(days=1)

    # 重置索引以确保索引与排序后的顺序相匹配
    monthly_averages_sorted.reset_index(drop=True, inplace=True)

    # print(monthly_averages_sorted)

    # 创建图形
    fig, ax = plt.subplots(figsize=(16, 6))

    # 绘制特征数据
    ax.plot(monthly_averages_sorted['time_period'],monthly_averages_sorted['real_time_user'], marker='o', label='进场用户', color='#4169e0')
    ax.plot(monthly_averages_sorted['time_period'], monthly_averages_sorted['leave_user'], marker='x', label='离线用户', color='#ff0000')
    ax.plot(monthly_averages_sorted['time_period'], monthly_averages_sorted['user_count'], marker='s', label='在线用户', color='#00cdd0')

    # 设置标题和标签
    # ax.set_title(f'每隔{int(time_interval[:-1])}分钟的特征变化', fontweight='bold')
    ax.set_ylabel('数量', fontweight='bold', fontproperties=font_prop,fontsize=20)
    ax.legend()

    # 添加文本
    ax.text(x=0.5, y=-0.1, s=f'(3)每隔{int(time_interval[:-1])}分钟的特征变化', ha='center', va='center', fontsize=20, transform=ax.transAxes)

    # 设置 x 轴标签
    ax.set_xlabel('时间区间', fontproperties=font_prop,fontsize=20,labelpad=30)

    ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

    ax.yaxis.set_major_locator(MaxNLocator(integer=True, nbins=5))  # 设置y轴刻度数量

    # 增加字体大小
    for label in ax.get_xticklabels():
        label.set_fontsize(20)  # 设置x轴刻度字体大小
    for label in ax.get_yticklabels():
        label.set_fontsize(20)  # 设置y轴刻度字体大小
    
    # 调整图例
    ax.legend(fontsize=20, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=3)  # 在正上方


    # 调整布局
    plt.tight_layout()

    # 保存图为文件
    plt.savefig(save_path)

    # 显示图形
    plt.show()

In [ ]:
def plot_feature_changes_part_1(filtered_data, time_interval='5T', save_path=None):
    """
    绘制每个等级的特征变化图，根据指定的时间间隔。

    Parameters:
    - filtered_data (pd.DataFrame): 包含数据的 DataFrame。
    - time_interval (str): 时间间隔，格式为 '5T', '10T', '30T' 等。
    - save_path (str): 图像保存路径。
    """
    # 创建数据副本，避免链式赋值问题
    filtered_data_n = filtered_data.copy()
    # 使用 .loc 方法进行赋值，避免链式赋值警告
    filtered_data_n.loc[:, 'time_period'] = filtered_data_n['minute'] // int(time_interval[:-1]) * int(time_interval[:-1])

    # 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
    average_features = filtered_data_n.groupby(['date', 'hour', 'time_period', 'room_id'])[['real_time_user', 'leave_user', 'user_count']].mean().reset_index()

    # 将同等级的直播间进行求和
    summed_features = average_features.groupby(['date', 'hour', 'time_period'])[['real_time_user', 'leave_user', 'user_count']].sum().reset_index()

    # 按月计算每个特征的平均值
    monthly_averages = summed_features.groupby(['hour', 'time_period']).agg({
        'real_time_user': 'mean',
        'leave_user': 'mean',
        'user_count': 'mean'
    }).reset_index()

    # 你想要的小时顺序
    desired_order = [19, 20, 21, 22, 23, 0, 1]

    # 将 hour 列转换为 Categorical 类型，并指定排序顺序
    monthly_averages['hour'] = pd.Categorical(monthly_averages['hour'], categories=desired_order, ordered=True)

    # 根据 hour 列和 time_period 列对 DataFrame 进行排序
    monthly_averages_sorted = monthly_averages.sort_values(['hour', 'time_period'])

    # 将 'time_period' 转换为实际时间格式
    monthly_averages_sorted['time_period_str'] = monthly_averages_sorted['time_period'].astype(int).astype(str).str.zfill(2) + ':00'
    monthly_averages_sorted['time_period'] = pd.to_datetime(monthly_averages_sorted['hour'].astype(str) + ':' + monthly_averages_sorted['time_period_str'], format='%H:%M:%S')

    # 对于 hour 为 0 或 1，日期加一天
    monthly_averages_sorted.loc[monthly_averages_sorted['hour'].isin([0, 1]), 'time_period'] = \
        monthly_averages_sorted['time_period'] + pd.DateOffset(days=1)

    # 重置索引以确保索引与排序后的顺序相匹配
    monthly_averages_sorted.reset_index(drop=True, inplace=True)

    # print(monthly_averages_sorted)

    # 创建图形
    fig, ax = plt.subplots(figsize=(16, 6))

    # 绘制特征数据
    ax.plot(monthly_averages_sorted['time_period'],monthly_averages_sorted['real_time_user'], marker='o', label='New Users', color='#4169e0')
    ax.plot(monthly_averages_sorted['time_period'], monthly_averages_sorted['leave_user'], marker='x', label='Offline Users', color='#ff0000')
    ax.plot(monthly_averages_sorted['time_period'], monthly_averages_sorted['user_count'], marker='s', label='Online Users', color='#00cdd0')

    # 设置标题和标签
    # ax.set_title(f'每隔{int(time_interval[:-1])}分钟的特征变化', fontweight='bold')
    ax.set_ylabel('Quantity',fontsize=20)
    ax.legend()

    # 添加文本
    ax.text(x=0.5, y=-0.1, s=f'(c) Features Change Every {int(time_interval[:-1])} Minutes', ha='center', va='center', fontsize=20, transform=ax.transAxes)

    # 设置 x 轴标签
    ax.set_xlabel('Time Interval',fontsize=20,labelpad=30)

    ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

    ax.yaxis.set_major_locator(MaxNLocator(integer=True, nbins=5))  # 设置y轴刻度数量

    # 增加字体大小
    for label in ax.get_xticklabels():
        label.set_fontsize(20)  # 设置x轴刻度字体大小
    for label in ax.get_yticklabels():
        label.set_fontsize(20)  # 设置y轴刻度字体大小
    
    # 调整图例
    ax.legend(fontsize=20, loc='upper right', ncol=1)  # 在右上方


    # 调整布局
    plt.tight_layout()

    # 保存图为文件
    plt.savefig(save_path)

    # 显示图形
    plt.show()

In [ ]:
plot_feature_changes_part(filtered_data, time_interval='1T',save_path='D:/研究生/毕业论文/代码/网络构建/直播平台网络模型/图片/时间粒度_1.svg')

In [ ]:
plot_feature_changes_part(filtered_data, time_interval='2T')

In [ ]:
plot_feature_changes_part(filtered_data, time_interval='3T',save_path='D:/研究生/毕业论文/草稿/图片/时间粒度_3.svg')

In [ ]:
plot_feature_changes_part(filtered_data, time_interval='10T',save_path='D:/研究生/毕业论文/草稿/图片/时间粒度_10.svg')

In [ ]:
plot_feature_changes_part(filtered_data, time_interval='12T')

In [ ]:
plot_feature_changes_part(filtered_data, time_interval='60T',save_path='D:/研究生/毕业论文/草稿/图片/时间粒度_60.svg')

In [ ]:
plot_feature_changes_part_1(filtered_data, time_interval='1T',save_path='D:/研究生/论文/期刊论文/小论文1/新期刊/草稿/figures/time_granularity_1.eps')

In [ ]:
plot_feature_changes_part_1(filtered_data, time_interval='10T',save_path='D:/研究生/论文/期刊论文/小论文1/新期刊/草稿/figures/time_granularity_10.eps')

In [ ]:
plot_feature_changes_part_1(filtered_data, time_interval='20T',save_path='D:/研究生/论文/期刊论文/小论文1/新期刊/草稿/figures/time_granularity_20.eps')

最终确认窗口大小设置为10.

In [ ]:
output_file = f'./处理数据/filtered_data.csv'
filtered_data.to_csv(output_file, index=False)

# 3.确认主播等级划分

In [ ]:
# 创建数据副本，避免链式赋值问题
filtered_data_n = filtered_data.copy()
# 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
# 同时获取每组 follower_count 的最大值
average_features = filtered_data_n.groupby(['room_id']).agg({
    'real_time_user': 'mean',
    'leave_user': 'mean',
    'user_count': 'mean',
    'follower_count': 'max',
    "like_count":'max'
}).reset_index()

In [ ]:
average_features.describe()

粉丝数小于10w可能是后面增大到10w造成的。

In [ ]:
# 霍普金斯统计量的计算函数
def hopkins(X):
    d = X.shape[1]  # 特征数量:3
    n = len(X)  # 样本数量:11516
    m = int(0.1 * n)  # 采样数量 (一般取样本数的10%):1152
    nbrs = NearestNeighbors(n_neighbors=1).fit(X)  # 最近邻计算

    random_sample_indices = random.sample(range(0, n), m)  # 从原数据中随机采样m个点
    u_distances = []  # 原数据点的最近邻距离
    w_distances = []  # 随机生成点的最近邻距离

    # 计算原数据点的最近邻距离
    for index in random_sample_indices:
        u_distances.append(
            nbrs.kneighbors([X[index]], 2, return_distance=True)[0][0][1]
        )

    # 随机生成点的最近邻距离
    for _ in range(m):
        random_point = [random.uniform(min(X[:, i]), max(X[:, i])) for i in range(d)]
        w_distances.append(nbrs.kneighbors([random_point], 1, return_distance=True)[0][0][0])

    # 计算霍普金斯统计量
    H = sum(w_distances) / (sum(w_distances) + sum(u_distances))
    return H

# 将数据转换为 numpy 数组
features = average_features[['real_time_user', 'leave_user', 'user_count']].values

# 计算霍普金斯统计量
hopkins_value = hopkins(features)
print(f"霍普金斯统计量: {hopkins_value}")

数据具有显著的聚类趋势，继续进行聚类分析。

## 3.1 kmeans

In [ ]:
sse = []
# silhouette_scores = []
k_values = range(1, 11)  # 通常可以尝试 2 到 10 个聚类
# 轮廓系数需要至少两个聚类才能计算。如果只有一个聚类，那么就没有其他聚类来比较，因此无法计算轮廓系数。

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features)
    sse.append(kmeans.inertia_)
    # silhouette_scores.append(silhouette_score(features, kmeans.labels_))

# 设置 Matplotlib 使用 ieee 样式
with plt.style.context(['ieee', 'no-latex']):
    # 创建图形
    plt.figure(figsize=(8, 6))

    # 绘制每个时刻的训练集大小
    plt.plot(k_values, sse, marker='o', linestyle='-', color='#fb0200', markersize=5,  linewidth=1)

    # 添加标签和标题
    plt.xlabel('簇数', fontproperties=font_prop,fontsize=14)
    plt.ylabel('误差平方和', fontproperties=font_prop,fontsize=14)
#     plt.title('Training Set Size Over Time', fontsize=16, fontweight='bold')
#     plt.legend()

    # 调整坐标轴刻度文本的大小
    plt.tick_params(axis='both', which='major', labelsize=14)

    # 设置网格线
#     plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    plt.tight_layout()

    # 保存图为PDF文件
    plt.savefig('D:/研究生/毕业论文/草稿/图片/拐点图.svg')

    # 显示图形
    plt.show()


# # 绘制肘部图
# plt.figure(figsize=(8, 6))

# # 绘制SSE图
# # plt.subplot(1, 2, 1)
# plt.plot(k_values, sse, marker='o')
# plt.xlabel('簇数', fontproperties=font_prop,fontsize=12)
# plt.ylabel('误差平方和', fontproperties=font_prop,fontsize=12)
# # plt.title('Elbow Method for Optimal k')
# plt.grid(True)

# # # 绘制轮廓系数图
# # plt.subplot(1, 2, 2)
# # plt.plot(k_values, silhouette_scores, marker='o')
# # plt.xlabel('Number of Clusters (k)')
# # plt.ylabel('Silhouette Score')
# # plt.title('Silhouette Scores for Optimal k')
# # plt.grid(True)

# plt.tight_layout()

# # 保存图形为 PDF
# plt.savefig('D:/研究生/毕业论文/代码/网络构建/直播平台网络模型/图片/拐点图.svg')
# plt.show()

In [ ]:
sse = []
# silhouette_scores = []
k_values = range(1, 11)  # 通常可以尝试 2 到 10 个聚类
# 轮廓系数需要至少两个聚类才能计算。如果只有一个聚类，那么就没有其他聚类来比较，因此无法计算轮廓系数。

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features)
    sse.append(kmeans.inertia_)
    # silhouette_scores.append(silhouette_score(features, kmeans.labels_))

# 设置 Matplotlib 使用 ieee 样式
with plt.style.context(['ieee', 'no-latex']):
    # 创建图形
    plt.figure(figsize=(8, 6))

    # 绘制每个时刻的训练集大小
    plt.plot(k_values, sse, marker='o', linestyle='-', color='#fb0200', markersize=5,  linewidth=1)

    # 添加标签和标题
    plt.xlabel('Clusters',fontsize=14)
    plt.ylabel('Sum of Squared Errors',fontsize=14)
#     plt.title('Training Set Size Over Time', fontsize=16, fontweight='bold')
#     plt.legend()

    # 调整坐标轴刻度文本的大小
    plt.tick_params(axis='both', which='major', labelsize=14)

    # 设置网格线
#     plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    plt.tight_layout()

    # 保存图为PDF文件
    plt.savefig('D:/研究生/论文/期刊论文/小论文1/草稿/图片/inflection_point.eps')

    # 显示图形
    plt.show()

在绘制的肘部图中，寻找 SSE 曲线的拐点，即曲线从快速下降转为缓慢下降的位置。

该拐点的 k 值就是推荐的最佳聚类数目。

k=4

In [ ]:
# 设置聚类数目，可以通过肘部法则或轮廓系数来选择最佳的k值
n_clusters = 2  # 这里假设为3，根据实际需要调整

# 使用 KMeans 进行聚类
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
average_features['cluster'] = kmeans.fit_predict(features)

# 查看每个聚类中的样本数量
print(average_features['cluster'].value_counts())

In [ ]:
cluster_data = average_features[average_features['cluster'] == 0]
cluster_data.describe()

In [ ]:
cluster_data = average_features[average_features['cluster'] == 1]
cluster_data.describe()

In [ ]:
cluster_data = average_features[average_features['cluster'] == 2]
cluster_data.describe()

In [ ]:
cluster_data = average_features[average_features['cluster'] == 3]
cluster_data.describe()

效果还可以

## 3.2 Birch

层次聚类的典型代表，天生就是为处理超大规模数据集而设计的，它利用一个树结构来快速聚类

In [ ]:
from sklearn.cluster import Birch

# 创建 BIRCH 模型，指定簇的数量
birch_model = Birch(n_clusters=2, threshold=0.9)

# 训练模型
birch_model.fit(features)

# 获取聚类标签
labels = birch_model.predict(features)

# 将聚类标签添加到数据中
average_features['cluster'] = labels


In [ ]:
# 查看每个聚类中的样本数量
print(average_features['cluster'].value_counts())

In [ ]:
cluster_data = average_features[average_features['cluster'] == 0]
cluster_data.describe()

In [ ]:
cluster_data = average_features[average_features['cluster'] == 1]
cluster_data.describe()

数据不标准化比标准化更好

## 3.3 DBSCAN

In [ ]:
# 计算距离矩阵
distance_matrix = pairwise_distances(features, metric='euclidean')

# 选择 k 值
k = 5  # 可以尝试不同的 k 值来观察效果

# 计算 k 距离
nbrs = NearestNeighbors(n_neighbors=k).fit(features)
distances, _ = nbrs.kneighbors(features)
k_distances = np.sort(distances[:, -1])

# 绘制 k 距离曲线
plt.figure(figsize=(12, 8))
plt.plot(np.arange(len(k_distances)), k_distances, marker='o', linestyle='-')
plt.title('k-Distance Graph')
plt.xlabel('Points Sorted by Distance')
plt.ylabel(f'{k}th Nearest Neighbor Distance')
plt.grid(True)
plt.show()


In [ ]:
# 使用 Kneedle 算法查找拐点
kneedle = KneeLocator(
    np.arange(len(k_distances)), k_distances, curve='convex', direction='increasing'
)

# 获取拐点位置
knee_point = kneedle.knee
knee_value = k_distances[knee_point]

# 标注拐点
plt.axvline(x=knee_point, color='r', linestyle='--', label=f'Elbow at index {knee_point}')
plt.axhline(y=knee_value, color='r', linestyle='--', label=f'k-Distance = {knee_value:.2f}')
plt.legend()
plt.show()

print(f"拐点位置: {knee_point}, 拐点对应的 k 距离值: {knee_value:.2f}")

In [ ]:
from sklearn.cluster import DBSCAN

# 创建 DBSCAN 模型
dbscan_model = DBSCAN(eps=789, min_samples=6)  # 根据数据和需求调整 eps 和 min_samples

# 训练模型
labels = dbscan_model.fit_predict(features)

# 将聚类标签添加到数据中
average_features['cluster'] = labels
#过滤噪声点：因为 DBSCAN 的噪声点被标记为 -1，我们在计算轮廓系数时忽略这些点。

# 查看每个聚类中的样本数量
print(average_features['cluster'].value_counts())

In [ ]:
cluster_data = average_features[average_features['cluster'] == 0]
cluster_data.describe()

In [ ]:
cluster_data = average_features[average_features['cluster'] == -1]
cluster_data.describe()

效果不好，数据量差异太大

总结：KMEANS最好

In [ ]:
average_features

## 3.4 平均聚类

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def balanced_kmeans(X, k, max_iterations=100):
    # 初始化质心位置
    centroids = initialize_centroids(X, k)
    
    for iteration in range(max_iterations):
        # 赋值步骤
        # 计算边权重（这里可以是距离或相似度等）
        edge_weights = compute_edge_weights(X, centroids)
        
        # 解决分配问题
        assignments = solve_assignment_problem(edge_weights)
        
        # 更新步骤
        new_centroids = update_centroids(X, assignments, k)
        
        # 检查质心位置是否变化
        if np.all(centroids == new_centroids):
            break
        
        centroids = new_centroids
    
    return assignments

def initialize_centroids(X, k):
    # 随机选择初始质心
    indices = np.random.choice(X.shape[0], k, replace=False)
    return X[indices]

def compute_edge_weights(X, centroids):
    # 计算边权重，这里使用欧氏距离
    n_samples = X.shape[0]
    k = centroids.shape[0]
    edge_weights = np.zeros((n_samples, n_samples))
    
    # 计算每个样本点到每个质心的距离
    distances = np.zeros((n_samples, k))
    for j in range(k):
        distances[:, j] = np.linalg.norm(X - centroids[j], axis=1)

    # 填充边权重矩阵
    for a in range(n_samples):
        centroid_index = (a % k)  # 使用 a mod k
        edge_weights[a, :] = distances[:, centroid_index]
    
    return edge_weights


def solve_assignment_problem(edge_weights):
    # 解决分配问题，使用匈牙利算法
    row_ind, col_ind = linear_sum_assignment(edge_weights)
    assignments = np.zeros(edge_weights.shape[0], dtype=int)
    assignments[row_ind] = col_ind
    return assignments

def update_centroids(X, assignments, k):
    # 更新质心位置
    new_centroids = np.zeros((k, X.shape[1]))
    
    for i in range(k):
        assigned_points = X[assignments == i]  # 选取分配到簇 i 的数据点
        if len(assigned_points) > 0:
            new_centroids[i] = np.mean(assigned_points, axis=0)  # 计算均值
        else:
            new_centroids[i] = np.random.rand(X.shape[1])  # 如果没有数据点，随机初始化质心
    
    return new_centroids


In [ ]:
# 使用示例
if __name__ == "__main__":
    X = features  
    k = 3  # 簇的数量
    
    # 运行平衡K-means算法
    assignments = balanced_kmeans(X, k)
    print("Cluster assignments:", assignments)

# 4.处理数据

In [ ]:
time_interval = '10T'
# 创建数据副本，避免链式赋值问题
filtered_data_n_1 = filtered_data.copy()
# 从 average_features 中只选择 'room_id' 和 'cluster' 列
average_features_subset = average_features[['room_id', 'cluster']]
# 按 'room_id' 列进行合并，只将 cluster 列添加到 filtered_data_n_1 中
merged_data = pd.merge(filtered_data_n_1, average_features_subset, on='room_id', how='left')

# 使用 .loc 方法进行赋值，避免链式赋值警告
merged_data.loc[:, 'time_period'] = merged_data['minute'] // (int(time_interval[:-1])) * int(time_interval[:-1])

# 计算每个直播间在每个时间段的平均特征，仅包括特定的特征
average_features_1 = merged_data.groupby(['date', 'hour', 'time_period', 'room_id', 'cluster'])[['real_time_user', 'leave_user', 'user_count']].mean().reset_index()

# 对每个小时内所有直播间的实时用户数取整，然后再求和
average_features_1['real_time_user'] = average_features_1['real_time_user'].round().astype(int)
average_features_1['leave_user'] = average_features_1['leave_user'].round().astype(int)
average_features_1['user_count'] = average_features_1['user_count'].round().astype(int)

# 将同等级的直播间进行求和
summed_features = average_features_1.groupby(['date', 'hour', 'time_period', 'cluster'])[['real_time_user', 'leave_user', 'user_count']].sum().reset_index()

# 获取所有唯一的等级
levels = summed_features['cluster'].unique()

# 对每个等级的数据进行提取
for level in levels:
    # 提取当前等级的数据
    level_data = summed_features[summed_features['cluster'] == level]

    # 删除不需要的列
    level_data = level_data.drop(columns=['cluster'])
    
    # 保存到 CSV 文件
    output_file = f'./处理数据/hourly_stats_level_{level}.csv'
    level_data.to_csv(output_file, index=False)
    print(f'Saved data for level {level} to {output_file}')


In [ ]:
data_folder = './处理数据'

# 处理每个级别的数据
for level in [0,1]:  # 假设你有4个级别，调整为实际需要的级别
    # 构建文件名
    file_name = f'hourly_stats_level_{level}.csv'
    file_path = os.path.join(data_folder, file_name)
    
    # 读取数据
    hourly_stats = pd.read_csv(file_path)
    
    # 重命名列
    hourly_stats.columns = ['date', 'hour', 'time_period', 'real_time_user', 'leave_user', 'user_count']

    # 确保数据按照日期、小时和时间段排序
    hourly_stats = hourly_stats.sort_values(by=['date', 'hour', 'time_period'])
    
    # 创建匹配后的数据
    merged_data = pd.DataFrame()

    # 定义时间间隔的步长
    time_intervals = list(range(0, 60, 10))  # 每隔10分钟
    for i in range(len(time_intervals) - 1):
        # 当前时间段和下一个时间段的分钟数
        current_time_period = time_intervals[i]
        next_time_period = time_intervals[i + 1]
        
        # 获取当前时间段和下一时间段的数据
        current_time_data = hourly_stats[hourly_stats['time_period'] == current_time_period]
        next_time_data = hourly_stats[hourly_stats['time_period'] == next_time_period]
        
        # 合并数据，确保同一天、同一小时的数据匹配
        merged_time_data_1 = pd.merge(current_time_data, next_time_data, on=['date', 'hour'], suffixes=('', '_t'))

        # 只保留匹配成功的行
        if not merged_time_data_1.empty:
            merged_data = pd.concat([merged_data, merged_time_data_1], axis=0)
    
    # 当前时间段和下一个时间段的分钟数
    current_time_period = time_intervals[5]
    next_time_period = time_intervals[0]

    # 获取当前时间段和下一时间段的数据
    current_time_data = hourly_stats[hourly_stats['time_period'] == current_time_period]
    next_time_data = hourly_stats[hourly_stats['time_period'] == next_time_period].copy()

    # 根据情况调整 next_time_data 的 hour
    # 合并跨日数据
    merged_time_data_2 = pd.DataFrame()  # 初始化空数据框
    merged_time_data_3 = pd.DataFrame()  # 初始化空数据框

    # 按照当前时间数据的 hour 分组处理
    grouped_current_time = current_time_data.groupby('hour', observed=False)
    print(grouped_current_time)
    for hour, group in grouped_current_time:
        if hour == 1:
            # 如果 hour == 1，跳过该组数据
            continue
        elif hour == 23:
            # 获取下一个小时的数据
            next_hour_data = next_time_data[next_time_data['hour'] == 0]
            if not next_hour_data.empty:
                # 合并数据
                merged_time_data_2 = pd.merge(group, next_hour_data, on=['date'], suffixes=('', '_t'))
                if not merged_time_data_2.empty:
                    merged_data = pd.concat([merged_data, merged_time_data_2], axis=0)
        else:
            # 获取当前小时 + 1 的下一时间段数据
            next_hour = hour + 1
            next_hour_data = next_time_data[next_time_data['hour'] == next_hour]
            if not next_hour_data.empty:
                # 合并数据
                merged_time_data_3 = pd.merge(
                    group, next_hour_data, 
                    on=['date'], suffixes=('', '_t')
                )
                # 只保留匹配成功的行
                if not merged_time_data_3.empty:
                    merged_data = pd.concat([merged_data, merged_time_data_3], axis=0)
    
    # 丢弃不必要的列（如 'time_period_t' 列）
    merged_data = merged_data.drop(columns=['time_period_t'])
    merged_data = merged_data.drop(columns=['hour_t'])
    merged_data = merged_data.drop(columns=['date'])

    # 根据 hour 和 time_period 生成新的索引
    index_mapping = {
    (19, 0): 0, (19, 10): 1, (19, 20): 2, (19, 30): 3, (19, 40): 4, (19, 50): 5,
    (20, 0): 6, (20, 10): 7, (20, 20): 8, (20, 30): 9, (20, 40): 10, (20, 50): 11, (21, 0): 12, (21, 10): 13, (21, 20): 14, (21, 30): 15, (21, 40): 16, (21, 50): 17, (22, 0): 18, (22, 10): 19, (22, 20): 20, (22, 30): 21, (22, 40): 22, (22, 50): 23, (23, 0): 24, (23, 10): 25, (23, 20): 26, (23, 30): 27, (23, 40): 28, (23, 50): 29, (0, 0): 30, (0, 10): 31, (0, 20): 32, (0, 30): 33, (0, 40): 34, (0, 50): 35}

    # 应用索引映射
    merged_data['index'] = merged_data.apply(lambda row: index_mapping.get((row['hour'], row['time_period']), -1), axis=1)


    # 将索引列移动到第一列
    merged_data = merged_data[['index'] + [col for col in merged_data.columns if col != 'index']]


    # 丢弃不必要的列（如 'time_period_t' 列）
    merged_data = merged_data.drop(columns=['hour'])
    merged_data = merged_data.drop(columns=['time_period'])


    # 保存处理后的数据
    output_file_path = os.path.join(data_folder, f'processed_hourly_stats_level_{level}.csv')
    merged_data.to_csv(output_file_path, index=False)
    print(f'Processed data saved for level {level} to {output_file_path}')

In [ ]:
# 定义数据文件夹路径
data_folder = './处理数据'

# 假设你有 3 个等级
levels = [0,1]  
level_change_rate_data = {}

for level in levels:
    # 构建文件路径
    file_path = os.path.join(data_folder, f'hourly_stats_level_{level}.csv')
    
    # 读取当前等级的数据
    level_data = pd.read_csv(file_path)

    # 你想要的小时顺序
    desired_order = [19, 20, 21, 22, 23, 0]

    # 将 hour 列转换为 Categorical 类型，并指定排序顺序
    level_data['hour'] = pd.Categorical(level_data['hour'], categories=desired_order, ordered=True)

    # 确保数据按照日期、小时和时间段排序
    level_data = level_data.sort_values(by=['date', 'hour', 'time_period'])
    
    # 按照 'hour', 'time_period' 分组，计算 'real_time_user' 特征的平均值
    avg_data = level_data.groupby(['hour', 'time_period'], observed=False)[['real_time_user']].mean().reset_index()

    
    # 计算变化率
    avg_data['next_real_time_user'] = avg_data['real_time_user'].shift(-1)
    avg_data['change_rate'] = (avg_data['next_real_time_user'] - avg_data['real_time_user']) / avg_data['real_time_user']
    
    # 处理时间跨小时的情况：当前的50分钟与下一小时的0分钟
    for idx, row in avg_data.iterrows():
        if row['time_period'] == 50:  # 如果是50分钟
            # 找到下一行的时间是下一小时的0分钟的数据
            next_index = idx + 1
            if next_index < len(avg_data):
                next_row = avg_data.iloc[next_index]
                if next_row['hour'] == (row['hour'] + 1) % 24 and next_row['time_period'] == 0:
                    avg_data.at[idx, 'change_rate'] = (
                        next_row['real_time_user'] - row['real_time_user']
                    ) / row['real_time_user']
            else:
                # 如果没有合适的下一行，则设置为NaN
                avg_data.at[idx, 'change_rate'] = float('nan')
    
    # 删除不需要的列
    avg_data = avg_data.drop(columns=['next_real_time_user'])

    avg_data = avg_data.drop(columns=['hour'])
    avg_data = avg_data.drop(columns=['time_period'])
    avg_data = avg_data.drop(columns=['real_time_user'])

    # 创建索引列
    avg_data['index'] = avg_data.reset_index().index+1
    
    # 将索引列移动到第一列
    avg_data = avg_data[['index'] + [col for col in avg_data.columns if col != 'index']]

    
    # 保存每个等级的变化率数据
    output_file = os.path.join(data_folder, f'change_rate_real_time_user_level_{level}.csv')
    avg_data.to_csv(output_file, index=False)
    print(f'Saved change rate data for level {level} to {output_file}')

    # 将结果存储到字典中，以便后续使用
    level_change_rate_data[level] = avg_data

In [ ]:
# 定义数据文件夹路径
data_folder = './处理数据'

# 假设你有 3 个等级
levels = [0,1]  
level_change_rate_data = {}

for level in levels:
    # 构建文件路径
    file_path = os.path.join(data_folder, f'hourly_stats_level_{level}.csv')
    
    # 读取当前等级的数据
    level_data = pd.read_csv(file_path)

    # 你想要的小时顺序
    desired_order = [19, 20, 21, 22, 23, 0]

    # 将 hour 列转换为 Categorical 类型，并指定排序顺序
    level_data['hour'] = pd.Categorical(level_data['hour'], categories=desired_order, ordered=True)

    # 确保数据按照日期、小时和时间段排序
    level_data = level_data.sort_values(by=['date', 'hour', 'time_period'])
    
    # 按照 'hour', 'time_period' 分组，计算 'real_time_user' 特征的平均值
    avg_data = level_data.groupby(['hour', 'time_period'], observed=False)[['real_time_user']].mean().reset_index()

    
    # 计算变化率
    avg_data['next_real_time_user'] = avg_data['real_time_user'].shift(-1)
    avg_data['change_rate'] = (avg_data['next_real_time_user'] - avg_data['real_time_user']) / avg_data['real_time_user']
    
    # 处理时间跨小时的情况：当前的50分钟与下一小时的0分钟
    for idx, row in avg_data.iterrows():
        if row['time_period'] == 50:  # 如果是50分钟
            # 找到下一行的时间是下一小时的0分钟的数据
            next_index = idx + 1
            if next_index < len(avg_data):
                next_row = avg_data.iloc[next_index]
                if next_row['hour'] == (row['hour'] + 1) % 24 and next_row['time_period'] == 0:
                    avg_data.at[idx, 'change_rate'] = (
                        next_row['real_time_user'] - row['real_time_user']
                    ) / row['real_time_user']
            else:
                # 如果没有合适的下一行，则设置为NaN
                avg_data.at[idx, 'change_rate'] = float('nan')
    
    # 删除不需要的列
    # avg_data = avg_data.drop(columns=['next_real_time_user'])

    # avg_data = avg_data.drop(columns=['hour'])
    # avg_data = avg_data.drop(columns=['time_period'])
    # avg_data = avg_data.drop(columns=['real_time_user'])

    # 创建索引列
    avg_data['index'] = avg_data.reset_index().index+1
    
    # 将索引列移动到第一列
    avg_data = avg_data[['index'] + [col for col in avg_data.columns if col != 'index']]

    
    # 保存每个等级的变化率数据
    output_file = os.path.join(data_folder, f'real_time_user_level_{level}.csv')
    avg_data.to_csv(output_file, index=False)
    print(f'Saved change rate data for level {level} to {output_file}')

    # 将结果存储到字典中，以便后续使用
    level_change_rate_data[level] = avg_data

# 5.描述结果

In [ ]:
paras=pd.read_csv('best_parameters_and_thresholds_l1.csv')

In [ ]:
paras.describe()

In [ ]:
paras.corr()

In [ ]:
l0=pd.read_csv('window_accuracy_results_l0.csv')
l1=pd.read_csv('window_accuracy_results_l1.csv')

In [ ]:
l0.describe()

In [ ]:
l1.describe()